# Eksperimantiranje

Prvo ćemo importati potrebne pakete i fiksirati _random seed_ kako bi kod bio reproducibilan.

In [1]:
from enum import Enum
from time import sleep

import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

random_state = 42
rng = np.random.RandomState(random_state)

Sljedeća funkcija služi generiranju uzorka jediničnih normalnih veličina $x$ i $y$ td.
\begin{align*}
y &= \beta x + \varepsilon, \ \ \varepsilon \sim N(0, \sigma^2) \\
\mathrm{Cov}(x_i, x_j) &= \sigma^2 > 0, \ \ i \neq j.
\end{align*}

In [2]:
def get_sample(beta: np.array, rho: float, sample_size: int):

    """Generates sample of X and y such that y = X*beta + eps, eps ~ N(0, sqrt(error_var))

    Args:
        beta (np.array): Linear transformation vector.
        rho (float): Covariance between covariates.

    Returns:
        sample: generated sample.
    """

    N = beta.shape[0]
    cov = rho * np.ones([N, N])
    cov = cov + np.diag(1-rho * np.ones(N))

    N = beta.shape[0]
    X = rng.multivariate_normal(mean = np.zeros(N), cov=cov, size=sample_size)

    error = rng.normal(loc=0, scale=1, size=X.shape[0])
    y = np.matmul(X, beta) + error
    
    return X, y

Brzinski _sanity check_: ako je $\rho = 0,$ tada je $$\mathrm{Var}(y) = \sum_{k=0}^N \beta_k^2 \mathrm{Var}(X_k) + \mathrm{Var}(\varepsilon),$$ što bi u slucaju $N = 3, \beta = (1, \dots, 1)$ moralo biti $4$. Zaista, za dovoljno velik uzorak imamo da je uzoračka varijanca $\approx 4$:

In [3]:
N = 3
beta = np.ones(3)
rho = 0
sample_size = 1000000

X, y = get_sample(beta=beta, rho=rho, sample_size=sample_size)
print(f'>>> Var(y) = {y.var(ddof=1)}')

>>> Var(y) = 3.9966141119858167


Nastavljamo definiranjem funkcija za treniranje LS, PCR i PLS modela, pri čemu potonja dva kao parametar primaju i broj glavnih komponenti koje koriste.

In [4]:
class Model(str, Enum):
    linreg = "linreg"
    pcr = "pcr"
    pls = "pls"

    @staticmethod
    def train(model_name: str, X: np.array, y: np.array, n_components: int | None = None):

        if model_name not in list(Model):
            raise ValueError(f'No such model. Available models are {list(Model)}')
        
        if model_name == Model.linreg:
            model = LinearRegression()
        
        elif model_name == Model.pcr:
            model = make_pipeline(PCA(n_components=n_components, random_state=random_state), LinearRegression())
        
        elif model_name == Model.pls:
            model = PLSRegression(n_components=n_components)
        
        model.fit(X, y)

        return model

Jedan razuman način validacije naših modela bio bi da koristeći distribucije iz kojih smo generirali podatke izračunamo populacijski $\beta$ pa za grešku modela uzmemo
koliko se njegov koeficijent razlikuje od populacijskog, tj. ako je nas model dan s $y = \hat{\beta}x$, njegovu grešku možemo računati kao
\begin{align*}
\mathrm{Err}(\mathrm{Model}) = \|\beta-\hat{\beta}\|.
\end{align*}
Međutim, kako je prilikom visoke korelacije kovarijata taj $\beta$ "nestabilan", mi ćemo umjesto toga testirati naše modele na velikom testnom uzorku. Preciznije, prvo ćemo izgenerirati jako velik uzorak, zatim trenirati model na njegovom malom dijelu, a na ostatku izračunati $R2,$ što ce nam biti primarna metrika za validaciju modela. Takav pristup ima dvije prednosti:
1. Metrike poput kvaratne greške i $R2$ su interpretabilnije od udaljenosti do stvarnog $\beta$.
2. Tako se stvari rade u praksi (jer ne znamo stvarne distribucije pa ni vrijednost populacijskog koeficijenta); istrenira se model na uzorku koji nam je dan, a zatim validira na testnom skupu pa ide u produkciju.



In [5]:
SAMPLE_SIZE = 50_000

In [6]:
def train_and_evaluate_all_models(X_train: np.array, X_test: np.array, y_train: np.array, y_test: np.array, n_components: int):
    
    """Trains LS, PLS and PCR models on X_train and y_test and evaluates them on X_test and y_test.

    Args:
        n_components (int): Number of relevant components to use for predicition in PCR and PLS.

    Returns:
        dict: Contains model names with their respective R2 values.
    """

    score_dict = {'R2': {}, 'MSE': {}}

    for model_name in list(Model):
        model = Model.train(model_name=model_name, X=X_train, y=y_train, n_components=n_components)
        score_dict['R2'][model_name.value] = model.score(X_test, y_test)
        preds_test = model.predict(X_test)
        score_dict['MSE'][model_name.value] = mean_squared_error(preds_test, y_test)

    return score_dict

Donji primjer pokazuje kako kod "skoro" nezavisnih kovarijata PLS bolje predviđa nego PCR. To je i očekivano, pogotovo ako je broj komponenti puno manji od $N$ jer tada PCA nužno gubi bitne informacije za predviđanje. S druge strane, PLS, rastavljajuci zavisnu varijablu skupa s nezavisnima, ne izgubi gotovo ništa te predviđa jednako dobro kao i linearna regresija, ali u puno manjoj dimenziji pa je stoga interpretabilniji od nje.

In [7]:
N = 300
train_sample_size = 1000
n_components = 5
rho= 0.1

beta = rng.normal(loc=0, scale=25, size=N)
X, y = get_sample(beta=beta, rho=rho, sample_size=SAMPLE_SIZE)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state, train_size=train_sample_size)

In [8]:
score_dict = train_and_evaluate_all_models(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, n_components=n_components)
score_dict

{'R2': {'linreg': 0.9999912424659761,
  'pcr': 0.003925273416792052,
  'pls': 0.9946948181360293},
 'MSE': {'linreg': 1.439361129263707,
  'pcr': 163711.75257618446,
  'pls': 871.943236292399}}

In [9]:
df = pd.DataFrame(score_dict)
df.index.name = 'model'
df = df.reset_index()
df

,model,R2,MSE
0,linreg,0.999991,1.439361
1,pcr,0.003925,163711.752576
2,pls,0.994695,871.943236


Sad ćemo za razne parametre provesti eksperiment i podatke o R2 spremiti u datoteku _scores.csv_.

In [11]:
SAMPLE_SIZE = 100_000
index_columns = ['M', 'N', 'n', 'rho']
columns = ['model', 'R2', 'MSE']
df = pd.DataFrame(None, columns=index_columns+columns)

Ns = [500, 100, 50, 10, 5]
loader = tqdm(Ns)

N_M_ratios = [10, 5, 3, 2, 1]
n_N_ratios = [0.01, 0.1, 0.25, 0.5]
rhos = [0.01, 0.1, 0.2, 0.5, 0.9, 0.99]

for N in loader:
    for rho in rhos:
        
        beta = rng.normal(loc=0, scale=1, size=N)
        X, y = get_sample(beta=beta, rho=rho, sample_size=SAMPLE_SIZE)

        for M in [N*x for x in N_M_ratios]:

            X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state, train_size=M)

            for n in [int(N*x) for x in n_N_ratios]:

                if n == 0:
                    continue

                score_dict = train_and_evaluate_all_models(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, n_components=n)
                score_df = pd.DataFrame(score_dict)
                score_df.index.name = 'model'
                score_df = score_df.reset_index()
                for col_name, value in zip(index_columns, [M, N, n, rho]):
                    score_df[col_name] = value
                print(df)
                df = pd.concat([df, score_df])
                loader.set_postfix(**score_dict['R2'] | {'rho': rho, 'N': N})
        
        score_df.to_csv('scores2.csv', index=False)

df.to_csv('scores2.csv', index=False)

  0%|          | 0/5 [00:04<?, ?it/s, N=500, linreg=0.997, pcr=0.0249, pls=0.997, rho=0.01]

Empty DataFrame
Columns: [M, N, n, rho, model, R2, MSE]
Index: []


  0%|          | 0/5 [00:12<?, ?it/s, N=500, linreg=0.997, pcr=0.0958, pls=0.997, rho=0.01]

      M    N  n   rho   model        R2         MSE
0  5000  500  5  0.01  linreg  0.997495    1.101917
1  5000  500  5  0.01     pcr  0.024885  428.943009
2  5000  500  5  0.01     pls  0.997442    1.125453


  0%|          | 0/5 [00:21<?, ?it/s, N=500, linreg=0.997, pcr=0.267, pls=0.997, rho=0.01] 

...

       M    N   n   rho   model         R2         MSE
0   5000  500   5  0.01  linreg   0.997495    1.101917
1   5000  500   5  0.01     pcr   0.024885  428.943009
2   5000  500   5  0.01     pls   0.997442    1.125453
0   5000  500  50  0.01  linreg   0.997495    1.101917
1   5000  500  50  0.01     pcr   0.095849  397.726742
..   ...  ...  ..   ...     ...        ...         ...
1      5    5   1  0.90     pcr  -0.520048    1.988598
2      5    5   1  0.90     pls  -0.630056    2.132516
0      5    5   2  0.90  linreg -12.647146   17.853842
1      5    5   2  0.90     pcr  -0.240935    1.623450
2      5    5   2  0.90     pls  -0.447742    1.894004

[1410 rows x 7 columns]
       M    N   n   rho   model        R2         MSE
0   5000  500   5  0.01  linreg  0.997495    1.101917
1   5000  500   5  0.01     pcr  0.024885  428.943009
2   5000  500   5  0.01     pls  0.997442    1.125453
0   5000  500  50  0.01  linreg  0.997495    1.101917
1   5000  500  50  0.01     pcr  0.095849  39

 80%|████████  | 4/5 [15:46<02:25, 145.92s/it, N=5, linreg=-11.6, pcr=-1.19, pls=-1.3, rho=0.99]      

       M    N   n   rho   model        R2         MSE
0   5000  500   5  0.01  linreg  0.997495    1.101917
1   5000  500   5  0.01     pcr  0.024885  428.943009
2   5000  500   5  0.01     pls  0.997442    1.125453
0   5000  500  50  0.01  linreg  0.997495    1.101917
1   5000  500  50  0.01     pcr  0.095849  397.726742
..   ...  ...  ..   ...     ...       ...         ...
1     50    5   2  0.99     pcr -0.049903    1.187679
2     50    5   2  0.99     pls -0.017846    1.151416
0     25    5   1  0.99  linreg -0.007109    1.139335
1     25    5   1  0.99     pcr  0.036253    1.090279
2     25    5   1  0.99     pls  0.036437    1.090072

[1419 rows x 7 columns]
       M    N   n   rho   model        R2         MSE
0   5000  500   5  0.01  linreg  0.997495    1.101917
1   5000  500   5  0.01     pcr  0.024885  428.943009
2   5000  500   5  0.01     pls  0.997442    1.125453
0   5000  500  50  0.01  linreg  0.997495    1.101917
1   5000  500  50  0.01     pcr  0.095849  397.726742
.. 

100%|██████████| 5/5 [15:46<00:00, 189.23s/it, N=5, linreg=-11.6, pcr=-.807, pls=-.942, rho=0.99]

       M    N   n   rho   model         R2         MSE
0   5000  500   5  0.01  linreg   0.997495    1.101917
1   5000  500   5  0.01     pcr   0.024885  428.943009
2   5000  500   5  0.01     pls   0.997442    1.125453
0   5000  500  50  0.01  linreg   0.997495    1.101917
1   5000  500  50  0.01     pcr   0.095849  397.726742
..   ...  ...  ..   ...     ...        ...         ...
1     10    5   2  0.99     pcr  -0.259403    1.424790
2     10    5   2  0.99     pls  -0.395749    1.579041
0      5    5   1  0.99  linreg -11.607016   14.262548
1      5    5   1  0.99     pcr  -1.193599    2.481659
2      5    5   1  0.99     pls  -1.295294    2.596708

[1437 rows x 7 columns]
